In [28]:
import os
import numpy
from PIL import Image
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize

## Frechet Inception Distance


In [43]:
def imagetensor(imagedir):
    """
    read image from dir and return np array for images
    """
    for i, im in enumerate(os.listdir(imagedir)):
        image= Image.open(os.path.join(imagedir, im))
        image= image.convert('RGB')
        if i == 0:
            images= np.expand_dims(np.array(image, dtype= float)/255, axis= 0)
        else:
            image= np.expand_dims(np.array(image, dtype= float)/255, axis= 0)
            images= np.append(images, image, axis= 0)
    return images

images1 = imagetensor('./images1')
images2 = imagetensor('./images2')

In [41]:
def scale_images(images, new_shape):
    """
    scale images into new shape
    """
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)
 
def calculate_fid(model, images1, images2):
    """
    calculate FID score for two sets of given images
    """
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid
 
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

print('Prepared', images1.shape, images2.shape)

images1 = images1.astype('float32')
images2 = images2.astype('float32')

images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)

images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)

fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

(10, 128, 128, 3)
Prepared (10, 128, 128, 3) (10, 128, 128, 3)
Scaled (10, 299, 299, 3) (10, 299, 299, 3)
FID (same): -0.000
FID (different): 12.328


## Reference
1. [How to implement the Frechet Inception Distance](https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/)